In [1]:
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from pprint import pprint

Using plaidml.keras.backend backend.


In [2]:
# load ascii text and covert to lowercase
filename = "FAHRENHEIT 451.txt"
raw_text = open(filename, 'r', encoding='utf-8').read() 
raw_text = raw_text.lower()
raw_text

'fahrenheit 451 by ray bradbury\nthis one, with gratitude, is for don congdon.\nfahrenheit 451:\nthe temperature at which book-paper catches fire and burns\npart i\nit was a pleasure to burn\nit was a special pleasure to see things eaten, to see things blackened and changed. with the brass nozzle in his fists, with this great python spitting its venomous kerosene upon the world, the blood pounded in his head, and his hands were the hands of some amazing conductor playing all the symphonies of blazing and burning to bring down the tatters and charcoal ruins of history. with his symbolic helmet numbered 451 on his stolid head, and his eyes all orange flame with the thought of what came next, he flicked the igniter and the house jumped up in a gorging fire that burned the evening sky red and yellow and black. he strode in a swarm of fireflies. he wanted above all, like the old joke, to shove a marshmallow on a stick in the furnace, while the flapping pigeon-winged books died on the porch 

In [3]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, '`': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49}


In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  251219
Total Vocab:  50


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 80
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  251139


In [7]:
print(dataX[1])

[24, 31, 41, 28, 37, 31, 28, 32, 43, 1, 14, 15, 11, 1, 25, 48, 1, 41, 24, 48, 1, 25, 41, 24, 27, 25, 44, 41, 48, 0, 43, 31, 32, 42, 1, 38, 37, 28, 7, 1, 46, 32, 43, 31, 1, 30, 41, 24, 43, 32, 43, 44, 27, 28, 7, 1, 32, 42, 1, 29, 38, 41, 1, 27, 38, 37, 1, 26, 38, 37, 30, 27, 38, 37, 9, 0, 29, 24, 31, 41]


In [8]:
print(dataY[1])

28


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [10]:
pprint(y)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)


In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]))) 
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-256.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [ ]:
# fit the model
model.fit(X, y, epochs=12, batch_size=8, callbacks=callbacks_list)

In [15]:
# load the network weights
filename = "weights-improvement-01-2.7781-256.hdf5" 
model.load_weights(filename) 
model.compile(loss='categorical_crossentropy', optimizer='adam')
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"") # generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  pprint(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  monstrous big. so, you see." he snapped it on. "montag," the tv set said, and
l "
'o'
' '
't'
'h'
'e'
' '
't'
'a'
'a'
'd'
' '
' '
'"'
'n'
'o'
' '
't'
' '
's'
'o'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'e'
' '
't'
'o'
'